In [1]:
import anndata as ad
import networkx as nx
import scanpy as sc
import pandas as pd
import numpy as np
import scglue
from matplotlib import rcParams
import os
os.chdir('/lustre/scratch/kiviaho/spatac/integrations/tonsilatlas/sampled_broad_celltypes_all')
date = '20221215'
multiple_experiments = False

In [2]:
rna_name = 'synthetic_spatial_rna'
atac_name = 'simulated_spots_atac'

rna = ad.read_h5ad(rna_name+'.h5ad')
atac = ad.read_h5ad(atac_name + '.h5ad')

# Copy raw counts into X, only if 
# For single cell
#rna.X = rna.raw.X.copy()
#atac.X = atac.raw.X.copy()

# For simulated data
rna.layers['counts'] = rna.X.copy()
atac.layers['counts'] = atac.X.copy()

In [3]:
scglue.data.get_gene_annotation(
    rna, gtf='../../../gencode.v42.annotation.gtf.gz',
    gtf_by="gene_name"
)

In [4]:
# Drop unannotated genes:
rna = rna[:,~rna.var.index.duplicated(keep='first')]
rna = rna[:,rna.var.dropna(subset=['chrom','chromStart','chromEnd']).index]

In [5]:
sc.pp.highly_variable_genes(rna, n_top_genes=2000, flavor="seurat_v3",span=1)
sc.pp.normalize_total(rna)
sc.pp.log1p(rna)
sc.pp.scale(rna)
sc.tl.pca(rna, n_comps=100, svd_solver="auto")

/home/ak431480/.conda/envs/spatac/lib/python3.8/site-packages/anndata/compat/_overloaded_dict.py:106: ImplicitModificationWarning: Trying to modify attribute `._uns` of view, initializing view as actual.
  self.data[key] = value


In [6]:
# Drop peak locations with zero peaks
atac = atac[:,~(atac.X.sum(axis=0)==0)]

# Extract coordinates
atac.var.rename(columns={'seqnames':'chrom','start':'chromStart','end':'chromEnd'},inplace=True) # just rename existing columns !b

""" split = atac.var_names.str.split(r"[:-]")
stacked_split = np.vstack(split).astype(str)
var_metadata = atac.var.copy()
chrom_info = pd.DataFrame({'chrom':stacked_split[:,0],
'chromStart':stacked_split[:,1],
'chromEnd':stacked_split[:,2]})
chrom_info.index = var_metadata.index
var_metadata = pd.concat([var_metadata,chrom_info],axis=1) """

' split = atac.var_names.str.split(r"[:-]")\nstacked_split = np.vstack(split).astype(str)\nvar_metadata = atac.var.copy()\nchrom_info = pd.DataFrame({\'chrom\':stacked_split[:,0],\n\'chromStart\':stacked_split[:,1],\n\'chromEnd\':stacked_split[:,2]})\nchrom_info.index = var_metadata.index\nvar_metadata = pd.concat([var_metadata,chrom_info],axis=1) '

In [8]:
sub_atac = atac[:5]
sub_rna = rna[:5]
# For some weird reason atac var column names revert back to the old ones. Correct them
sub_atac.var.rename(columns={'seqnames':'chrom','start':'chromStart','end':'chromEnd'},inplace=True) # 

In [13]:

# Moved guidance graph construction to BEFORE lsi calculation, since it should be using HVGs (which it gets from the RNA)
guidance = scglue.genomics.rna_anchored_guidance_graph(sub_rna, sub_atac)
scglue.graph.check_graph(guidance, [sub_rna, sub_atac])
nx.write_graphml(guidance, "guidance_graph_"+atac_name+"_"+rna_name+"_"+date+".graphml.gz")

window_graph:   0%|          | 0/23514 [00:00<?, ?it/s]

/home/ak431480/.conda/envs/spatac/lib/python3.8/site-packages/scglue/genomics.py:571: ImplicitModificationWarning: Trying to modify attribute `.var` of view, initializing view as actual.
  other.var["highly_variable"] = [


[INFO] check_graph: Checking variable coverage...
[INFO] check_graph: Checking edge attributes...
[INFO] check_graph: Checking self-loops...
[INFO] check_graph: Checking graph symmetry...
[INFO] check_graph: All checks passed!


In [17]:
atac.var = sub_atac.var

MemoryError: Unable to allocate 27.2 GiB for an array with shape (50000, 146076) and data type float32

: 

In [14]:
nx.write_graphml(guidance, "guidance_graph_"+atac_name+"_"+rna_name+"_"+date+".graphml.gz")

In [ ]:

scglue.data.lsi(atac,n_components=100) # This might take up too much space. If so, preprocess on a node using (xx-glue-preprocessing.py)

In [ ]:
rna.write("preprocessed_"+rna_name+"_"+date+".h5ad", compression="gzip")

In [ ]:
atac.write('preprocessed_'+atac_name+'_'+date+".h5ad", compression="gzip")
try:
    rna.write("preprocessed_"+rna_name+"_"+date+".h5ad", compression="gzip")
except:
    rna.var = rna.var.drop(columns='artif_dupl')
    rna.write("preprocessed_"+rna_name+"_"+date+".h5ad", compression="gzip")
    

In [ ]:
rna

In [ ]:
atac